# Bronze Layer - Setup

Este notebook cria o schema e as tabelas Bronze no Databricks.

**Nota:** Os dados são ingeridos do PostgreSQL local via script `scripts/ingestion/databricks_bronze_ingestion.py`. Este notebook apenas prepara a estrutura.


## 1) Configuração

Defina o catalog e schema. Se não usar Unity Catalog, defina `CATALOG = None`.


In [0]:
# ===== Parâmetros =====
CATALOG = "manufatura_lakehouse"  # ou None se não usar Unity Catalog
SCHEMA  = "bronze"

# ===== Nome totalmente qualificado de tabela =====
def fqtn(table):
    if CATALOG:
        return f"`{CATALOG}`.`{SCHEMA}`.`{table}`"
    else:
        return f"`{SCHEMA}`.`{table}`"

# ===== Criar schema/database =====
if CATALOG:
    spark.sql(f"CREATE CATALOG IF NOT EXISTS `{CATALOG}`")
    spark.sql(f"CREATE SCHEMA  IF NOT EXISTS `{CATALOG}`.`{SCHEMA}`")
    spark.sql(f"USE CATALOG `{CATALOG}`")
else:
    spark.sql(f"CREATE DATABASE IF NOT EXISTS `{SCHEMA}`")

print(f"✅ Schema {fqtn('')} criado ou já existe")


In [0]:
%sql
-- DROP TABLE IF EXISTS bronze.equipment_master;
DROP TABLE IF EXISTS bronze.iot_sensor_readings;
-- DROP TABLE IF EXISTS bronze.production_orders;
-- DROP TABLE IF EXISTS bronze.maintenance_orders;
-- DROP TABLE IF EXISTS bronze.quality_inspections;


## 2) Criar tabelas Bronze

Todas as colunas são STRING para simular dados brutos heterogêneos.


In [0]:
%sql
-- equipment_master
CREATE TABLE IF NOT EXISTS bronze.equipment_master (
    equipment_id      STRING,
    equipment_name    STRING,
    equipment_type    STRING,
    location          STRING,
    installation_date STRING,
    manufacturer      STRING,
    model             STRING,
    status            STRING,
    last_update_date  STRING
)
USING DELTA
TBLPROPERTIES (
    'delta.autoOptimize.optimizeWrite' = 'true',
    'delta.autoOptimize.autoCompact' = 'true'
);


In [0]:
%sql
-- iot_sensor_readings
CREATE TABLE IF NOT EXISTS bronze.iot_sensor_readings (
    reading_id        STRING,
    equipment_id      STRING,
    sensor_id         STRING,
    sensor_type       STRING,
    reading_value     STRING,
    reading_timestamp STRING,
    unit              STRING
)
USING DELTA
TBLPROPERTIES (
    'delta.autoOptimize.optimizeWrite' = 'true',
    'delta.autoOptimize.autoCompact' = 'true'
);


In [0]:
%sql
-- production_orders
CREATE TABLE IF NOT EXISTS bronze.production_orders (
    production_order_id STRING,
    equipment_id        STRING,
    product_id          STRING,
    planned_start       STRING,
    planned_end         STRING,
    actual_start        STRING,
    actual_end          STRING,
    planned_quantity    STRING,
    actual_quantity     STRING,
    status              STRING,
    last_update         STRING
)
USING DELTA
TBLPROPERTIES (
    'delta.autoOptimize.optimizeWrite' = 'true',
    'delta.autoOptimize.autoCompact' = 'true'
);


In [0]:
%sql
-- maintenance_orders
CREATE TABLE IF NOT EXISTS bronze.maintenance_orders (
    maintenance_order_id STRING,
    equipment_id         STRING,
    maintenance_type     STRING,
    scheduled_start      STRING,
    scheduled_end        STRING,
    actual_start         STRING,
    actual_end           STRING,
    technician_id        STRING,
    status               STRING,
    priority             STRING,
    description          STRING,
    last_update          STRING
)
USING DELTA
TBLPROPERTIES (
    'delta.autoOptimize.optimizeWrite' = 'true',
    'delta.autoOptimize.autoCompact' = 'true'
);


In [0]:
%sql
-- quality_inspections
CREATE TABLE IF NOT EXISTS bronze.quality_inspections (
    inspection_id       STRING,
    production_order_id STRING,
    equipment_id        STRING,
    inspection_type     STRING,
    inspection_date     STRING,
    inspector_id        STRING,
    passed              STRING,
    failed_quantity     STRING,
    total_quantity      STRING,
    defect_codes        STRING,
    notes               STRING,
    last_update         STRING
)
USING DELTA
TBLPROPERTIES (
    'delta.autoOptimize.optimizeWrite' = 'true',
    'delta.autoOptimize.autoCompact' = 'true'
);


## 3) Validação

Verificar se as tabelas foram criadas corretamente.


In [0]:
%sql
-- Listar todas as tabelas Bronze
SHOW TABLES IN bronze;


In [0]:
%sql
-- Verificar contagem de registros (deve estar vazio inicialmente)
SELECT 
    'equipment_master' as tabela, COUNT(*) as registros FROM bronze.equipment_master
UNION ALL
SELECT 'iot_sensor_readings', COUNT(*) FROM bronze.iot_sensor_readings
UNION ALL
SELECT 'production_orders', COUNT(*) FROM bronze.production_orders
UNION ALL
SELECT 'maintenance_orders', COUNT(*) FROM bronze.maintenance_orders
UNION ALL
SELECT 'quality_inspections', COUNT(*) FROM bronze.quality_inspections
ORDER BY tabela;


## Próximos passos

1. Execute o script de ingestão local para popular as tabelas Bronze:
   ```bash
   uv run ingest --table all
   ```

2. Após a ingestão, execute o notebook `silver_layer.ipynb` para processar os dados.
